In [1]:
import groq
import os
from dotenv import load_dotenv
import dspy
import numpy as np
from dspy.evaluate.metrics import answer_exact_match
import pandas as pd
from IPython.core.display import Markdown
from dspy import Example
from dspy.teleprompt import BootstrapFewShot
from dspy.teleprompt import BootstrapFewShotWithRandomSearch
from dspy.evaluate import Evaluate
from module_v002 import FullLLMChain
from optimize import passage_similarity_metric, custom_evaluation_function, similar_score_metric, evaluate_expectations_metric, create_dataframe_with_validation_results
import json
from data.preprocess import create_dspy_examples_train_test_validation_sets
load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")


llama3_8b = dspy.OllamaLocal(model = "llama3:8b",
                             temperature = 0,
                             max_tokens = 800)

gpt35turbo = dspy.OpenAI(model = "gpt-3.5-turbo-0125",
                         api_key = openai_api_key,
                         temperature = 0,
                         max_tokens = 800,
                         model_type = "chat")



# Import Data and Create Training, Testing, Validation Splits

In [2]:
data = pd.read_excel("data/300_snippets_transcripts_all_labeled_v002.xlsx")

train_examples, test_examples, validation_examples = create_dspy_examples_train_test_validation_sets(data)

# Run with the Unoptimzed GPT Model

In [7]:
dspy.settings.configure(lm=llama3_8b)
full_llm_chain_unoptimized = FullLLMChain()

In [8]:
results_unoptimized = create_dataframe_with_validation_results(
    validation_examples=validation_examples[0:2],
    llm_chain=full_llm_chain_unoptimized
)

0
1


In [5]:
metric_unoptimized = results_unoptimized["Evaluation_Score"].mean()
metric_unoptimized

0.0

In [11]:
results_unoptimized["Rationale"][0]

'Country Keyword: German\nCountry Role: \nExcerpt:'

# Run with the Optimized GPT Model

In [ ]:
dspy.settings.configure(lm=llama3_8b)

full_llm_chain_optimized = FullLLMChain()
full_llm_chain_optimized.load(path="optimized_llm_chains/optimized_llama_chain_v002.json")

In [ ]:
results_optimized = create_dataframe_with_validation_results(
    validation_examples=validation_examples[0:2],
    llm_chain=full_llm_chain_unoptimized,
)

In [ ]:
metric_optimized = results_unoptimized["Evaluation_Score"].mean()
metric_optimized

In [ ]:
results_optimized.head()